In [ ]:
# This is a notbeook as beatiful as a diamond.

In [ ]:
# import os
# import openai
import tensorflow as tf
import keras
from keras.datasets import imdb
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import numpy as np
from keras import models
from keras import layers
import jupyter

# Don't forget to normalize the dataset - hot encoding
# scalars stay normal columns, categories get different columns

# ln(price) / invert

# max / min normalization the dataset

# from sentence_transformers import SentenceTransformer

def short_roberta(sentences):
    sentences = ["This is an example sentence", "Each sentence is converted"]

    model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')
    embeddings = model.encode(sentences)
    print(embeddings)
    return embeddings


# roberta fully copied from https://huggingface.co/sentence-transformers/all-roberta-large-v1
def long_roberta(sentences):
    # Mean Pooling - Take attention mask into account for correct averaging
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Sentences we want sentence embeddings for
    # sentences = ['This is an example sentence', 'Each sentence is converted']

    # Load model from HuggingFace Hub
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-roberta-large-v1')
    model = AutoModel.from_pretrained('sentence-transformers/all-roberta-large-v1')

    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings


def as_book():
    def vectorize_sequences(sequences, dimension=10000):
        results = np.zeros((len(sequences), dimension))
        for i, sequence in enumerate(sequences):
            results[i, sequence] = 1.
        return results

    (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=1000)
    x_train = vectorize_sequences(train_data)
    y_test = vectorize_sequences(test_data)
    return x_train, y_test


def get_data():
    def decode(data, reverse_word_index):
        decoded_train_data = []
        for review in data:
            decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in review])
            decoded_train_data.append(decoded_review)
        return decoded_train_data

    word_index = imdb.get_word_index()
    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=100)
    train_data = train_data[:int(round(len(train_data) / 2000))]
    train_labels = train_labels[:int(round(len(train_labels) / 2000))]
    test_data = test_data[:int(round(len(test_data) / 2000))]
    test_labels = test_labels[:int(round(len(test_labels) / 2000))]

    print(decode(train_data, reverse_word_index)[0])
    x_train = long_roberta(decode(train_data, reverse_word_index)).numpy()
    x_test = long_roberta(decode(test_data, reverse_word_index)).numpy()
    y_train = np.asarray(train_labels).astype('float32')
    y_test = np.asarray(test_labels).astype('float32')

    return x_train, x_test, y_train, y_test
    # sentences = ['This is an example sentence', 'Each sentence is converted']
    # return sentences


def main():
    x_train, x_test, y_train, y_test = get_data()
    '''
    x_train = x_train[:len(x_train)/20]
    x_test = x_test[:len(x_test)/20]
    y_train = y_train[:len(y_train)/20]
    y_test = y_test[:len(y_test)/20]
    '''
    print(type(x_train))
    print(len(x_train))
    print(type(x_train[0]))
    print(len(x_train[0]))
    print(x_train[0])

    x_val = x_train[:500]
    partial_x_train = x_train[500:]
    y_val = y_train[:500]
    partial_y_train = y_train[500:]

    model = models.Sequential()
    model.add(layers.Dense(16, activation='relu', input_shape='1024'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    print('start compiling')

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(partial_x_train, partial_y_train, epochs=4, batch_size=256, validation_data=(x_val, y_val))

    plot(history)


def plot(history):
    import matplotlib.pyplot as plt

    history_dict = history.history
    loss_values = history_dict["loss"]
    val_loss_values = history_dict["val_loss"]
    epochs = range(1, len(loss_values) + 1)
    plt.plot(epochs, loss_values, "bo", label="Training loss")
    plt.plot(epochs, val_loss_values, "b", label="Validation loss")
    plt.title("Training and validation loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()


if __name__ == '__main__':
    main()


In [ ]:
import numpy as np
import math
import jupyter
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch.nn.functional as F
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
import os
import pandas as pd
from torchvision.io import read_image
from sklearn.preprocessing import MinMaxScaler


class CustomDiamondDataset(Dataset):
    def __init__(self, data, prices):
        self.x = data
        self.y = prices
        self.length = self.x.shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def main():
    data = pd.read_csv('diamonds.csv')
    data, prices = normalize_data(data)
    data = torch.tensor(data, dtype=torch.float32)
    prices = torch.tensor(prices, dtype=torch.float32)
    dataset = CustomDiamondDataset(data, prices)
    print(len(data[0]))
    dataloader = DataLoader(dataset=dataset, batch_size=40, shuffle=True, num_workers=2)
    model = NeuralNetwork(len(data[0]))
    print(type(data))
    print(type(prices))
    loss = nn.MSELoss
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    for epoch in range(100):
        for step, (input, label) in enumerate(dataloader):
            y_pred = model(input)
            l = loss(label, y_pred)
            l.backward()
            optimizer.step()
            optimizer.zero_grad()
    '''
    n_samples, n_features = data.shape
    input_size = n_features
    model = nn.Linear(input_size, 1) # correct this !
    prices = torch.tensor(prices, dtype=torch.float32)

    loss = nn.MSELoss
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

    for i in range(100):
        y_pred = model(X)
        l = loss(Y, y_pred)
        l.backward()
        optimizer.step()
        optimizer.zero_grad()
    # output = model(input).item()
    # print(type(data))
    # print(prices)
    # dataset = np.loadtxt('pima-indians-diabetes.csv', delimiter=',')
    '''


# both robertas fully copied from https://huggingface.co/sentence-transformers/all-roberta-large-v1
def short_roberta(sentences):
    sentences = ["This is an example sentence", "Each sentence is converted"]

    model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')
    embeddings = model.encode(sentences)
    print(embeddings)
    return embeddings


def long_roberta(sentences):
    # Mean Pooling - Take attention mask into account for correct averaging
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Sentences we want sentence embeddings for
    # sentences = ['This is an example sentence', 'Each sentence is converted']

    # Load model from HuggingFace Hub
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-roberta-large-v1')
    model = AutoModel.from_pretrained('sentence-transformers/all-roberta-large-v1')

    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings


def invert_min_max(val, min, max):
    return (max-min) * val + min

def normalize_data(data):
    # don't forget to normalize data - hot encoding
        # max / min normalization the dataset
    # scalars stay normal columns, categories get different columns
    # put price on a logarythmic scale

    '''
    lets build a tensor with the following dimensions:
        carat
        *cut* (hot encoding)
            ideal
            premium
            good
            very good
            fair
        colour (auch hot encoding)
        clarity (dito)
        depth
        table
        price
        x
        y
        z
    then min max the full thing.
    '''
    def onehot():
        nb_classes = 6
        arr = np.array([[2, 3, 4, 0]])
        targets = arr.reshape(-1)
        one_hot_targets = np.eye(nb_classes)[targets]
        return one_hot_targets

    onehot()
    np_data = data.to_numpy()

    cut_index = {'Fair':0, 'Good':1, 'Very Good':2, 'Premium':3, 'Ideal':4}
    colour_index = {'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D':6}
    # clarity: (I1(worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF(best))
    clarity_index = {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1':6, 'IF': 7}
    indeces = [cut_index, colour_index, clarity_index]


    # carat, cut (5), colour (7), clarity (8), depth, table, price, x, y
    new_array = []
    prices = []
    for i, diamond in enumerate(np_data):
        diamond = diamond[1:]
        new_diamond = [diamond[0]]
        for j in range(3):
            index = indeces[j][diamond[j+1]]
            zeros = [0]*len(indeces[j].keys())
            zeros[index] = 1
            for k in zeros:
                new_diamond.append(k)
        for j in [4, 5, 7, 8]:
            new_diamond.append(diamond[j])
        new_array.append(new_diamond)
        prices.append(math.log(diamond[6]))

    scaler = MinMaxScaler()
    data = pd.DataFrame(new_array)
    prices = pd.DataFrame(prices)
    normalized_data = scaler.fit_transform(data)
    normalized_prices = scaler.fit_transform(prices)

    return normalized_data, normalized_prices


if __name__ == '__main__':
    main()

